# 1. Introduction

The purpose of this notebook is to perform initial cleaning of the historical weather data, and to export the cleaned data to a csv file.

# 2. Setup & Data Load

Import required modules and packages:

In [41]:
# import pandas for data analysis
import pandas as pd

# import convert_timestamp for various timestamp conversion functions
from datetime import datetime

Set the max number of columns & rows to display:

In [42]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 500)

Weather data is loaded from a csv:

In [43]:
df_weather = pd.read_csv('../data_analysis/initial_data/his_weather.csv', sep=",",index_col=0, na_values=['\\N'])

In [44]:
df_weather.head(5)

,dt,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds,weather_id,weather_main,weather_desc
0,1514764800,4.15,-6.49,3.84,5.79,990,87,12.86,240,40,520,Rain,light intensity shower rain
1,1514768400,4.14,-5.79,3.65,5.86,991,87,11.83,240,75,520,Rain,light intensity shower rain
2,1514772000,4.61,-5.77,3.85,5.99,990,81,12.35,240,40,802,Clouds,scattered clouds
3,1514775600,4.64,-5.73,4.00,6.14,990,81,12.35,240,40,802,Clouds,scattered clouds
4,1514779200,5.04,-4.91,4.11,6.22,990,81,11.83,240,40,802,Clouds,scattered clouds


In [45]:
rows = df_weather.shape[0]
cols = df_weather.shape[1]
print()
print("Before any data cleaning, the dataframe contains", rows, "rows and", cols, "columns.")
print()


Before any data cleaning, the dataframe contains 8760 rows and 13 columns.



# 3. Check for Duplicate Rows & Columns

In [46]:
print()
print('Duplicate rows:', df_weather.duplicated()[df_weather.duplicated() == True].shape[0])
print('Duplicate columns:',df_weather.columns.size - df_weather.columns.unique().size)


Duplicate rows: 0
Duplicate columns: 0


There are no duplicate rows or columns so nothing needs to be dropped here.

# 4. Assign Features as Continuous or Categorical

Print 5 rows from the dataframe:

In [47]:
df_weather.dtypes

dt                int64
temp            float64
feels_like      float64
temp_min        float64
temp_max        float64
pressure          int64
humidity          int64
wind_speed      float64
wind_deg          int64
clouds            int64
weather_id        int64
weather_main     object
weather_desc     object
dtype: object

Assign categorical and continuous features:

In [48]:
# Select columns containing continuous data 
# This is done by selecting columns with a numeric type - float64 or int64
continuous_columns = df_weather.select_dtypes(['float64', 'int64']).columns

In [49]:
# Select columns containing categorical data

categorical_columns = df_weather[['dt', 'weather_main', 'weather_desc','weather_id']].columns

# Convert data type to 'Category' for these columns
for column in categorical_columns:
    df_weather[column] = df_weather[column].astype('category')

# 5 Check for Constant Features

In [50]:
# Print details for the categorical columns
df_weather[categorical_columns].describe().T

,count,unique,top,freq
dt,8760,8760,1546297200,1
weather_main,8760,8,Clouds,6760
weather_desc,8760,24,broken clouds,4103
weather_id,8760,23,803,4103


In [51]:
# Print details for the continuous columns
df_weather[continuous_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
temp,8760.0,10.082808,5.383556,-4.96,6.1375,9.760,13.97,25.67
feels_like,8760.0,5.850083,6.927927,-16.08,0.9100,5.455,10.92,24.77
temp_min,8760.0,8.871974,4.940494,-6.00,5.3475,8.875,12.78,19.97
temp_max,8760.0,11.244189,5.066155,-3.42,7.4400,11.000,15.00,27.00
pressure,8760.0,1012.623973,11.755886,979.00,1005.0000,1014.000,1021.00,1041.00
humidity,8760.0,80.497945,13.135822,25.00,72.0000,82.000,93.00,100.00
wind_speed,8760.0,5.143895,2.752965,0.34,3.0900,4.630,6.69,18.52
wind_deg,8760.0,189.524429,91.359421,0.00,120.0000,210.000,260.00,360.00
clouds,8760.0,60.247945,23.829674,0.00,40.0000,75.000,75.00,100.00


# 6. Check for Missing Data

Investigate rows with missing data:

In [52]:
df_weather[df_weather.isnull().values==True]

,dt,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds,weather_id,weather_main,weather_desc


non of missing data

# 7. weather data 

https://openweathermap.org/weather-conditions   
this is the meaning of weather ID

In [53]:
# from the website, i just recognize the extreme weather manually.
extreme_weather_id=[202,212,232,502,503,504,602,622,701,711,721,731,741,751,761,762,771,781]

In [54]:
# extreme weather 
def extre_weather(col,count=0):
    for i, row in df_weather.iterrows():
        if row[col] in extreme_weather_id:
            df_weather.at[i,'extre_weather']=1
            count+=1
        else:
            df_weather.at[i,'extre_weather']=0
    return count

count=extre_weather('weather_id')
df_weather['extre_weather'] = df_weather['extre_weather'].astype('int64')
count

150

there is 150 extreme weather in 2018.weather descrip is highly relevent to weatherID so i think we just need to keep one. weatherID can be used to judge whether the weather is extreme

In [55]:
# select all rows where irain is not 0
df_weather['weather_main'].value_counts()

Clouds     6760
Rain       1003
Drizzle     480
Clear       252
Snow        116
Fog          80
Mist         68
Smoke         1
Name: weather_main, dtype: int64

most of the weather is cloudy

# 9. timestamp convert

In [56]:
# timestamp convertion 
def convert_timestamp():
    for i, row in df_weather.iterrows():
        df_weather.at[i,'datetime']=datetime.fromtimestamp(row['dt'])
    return 

convert_timestamp()
df_weather['datetime'] = df_weather['datetime'].astype('datetime64[ns]')

# 10.drop reduntant features

In [57]:
df_weather.columns.str.strip()

Index(['dt', 'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'clouds', 'weather_id',
       'weather_main', 'weather_desc', 'extre_weather', 'datetime'],
      dtype='object')

In [68]:
df_weather.head(1)

,dt,temp,temp_max,pressure,humidity,wind_speed,wind_deg,clouds,weather_main,extre_weather,datetime
0,1514764800,4.15,5.79,990,87,12.86,240,40,Rain,0,2018-01-01


In [70]:
df_weather.drop(columns=['feels_like','temp_min','weather_id','weather_desc'],inplace=True)

KeyError: "['feels_like' 'temp_min' 'weather_id' 'weather_desc'] not found in axis"

In [71]:
df_weather.drop(columns=['temp_max'],inplace=True)

KeyError: "['temp_max'] not found in axis"

# 10. Export the Cleaned Data

In [72]:
df_weather.to_csv('../data_analysis/clean_data/his_weather_data.csv', index=False)

Import the data when required:

# 11. Data Quality Plan

| Feature | Data Quality Issue | Handling Strategy |
|-------------------------|----------------------|------------------------------|
| weather_desc | redundant feature | Drop feature |
| feels_like | redundant feature | Drop feature |
| temp_min | redundant feature | Drop feature |
| weather_id | redundant feature | Drop feature |
| temp_max | redundant feature | Drop feature |
| pressure | redundant feature | Drop feature |
| wind_deg | redundant feature | Drop feature |
| datetime |  | created new features |
| extreme_weather |  | created new features |
